In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!pip install geopy --user

%pip search geopy

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans


import folium 

print('Libraries imported.')

tornado-geopy (0.1.0)         - tornado-geopy is an asynchronous version of
                                the awesome geopy library.
geopy (2.0.0)                 - Python Geocoding Toolbox
  INSTALLED: 2.0.0 (latest)
swisslandstats-geopy (0.8.0)  - Python for the land statistics datasets from
                                the SFSO
pivotal-geopy (1.0.0)         - Python Geocoding Toolbox
Note: you may need to restart the kernel to use updated packages.
Libraries imported.


In [2]:
dataframe = pd.read_csv(r'/resources/data/Playschools.csv',encoding = "ISO-8859-1")
dataframe.head(10)

,Nursery Name,Borough
0,YMCA Thames Gateway Romford Pre School,Barking & Dagenham
1,Playland Pre-School,Barking & Dagenham
2,Cornerstone Academy,Barking & Dagenham
3,Starlings Nursery,Barnet
4,Hendon Prep Preschool,Barnet
5,Lubavitch of Edgware Kindergarten,Barnet
6,Nagila Preschool,Barnet
7,The Four Seasons Montessori,Barnet
8,The Wood Street Montessori,Barnet
9,Alban Pre-school,Barnet


I have the geocoordinates of all the above schools from Google Maps and I'm loading it as a separate Dataframe!

In [3]:
London = pd.read_csv(r'/resources/data/London.csv',encoding = "ISO-8859-1")
London.head(11)

,Nursery Name,Borough,Latitude,Longitude
0,YMCA Thames Gateway Romford Pre School,Barking & Dagenham,51.564939,0.182207
1,Playland Pre-School,Barking & Dagenham,51.538405,0.079790
2,Cornerstone Academy,Barking & Dagenham,51.555600,0.152300
3,Starlings Nursery,Barnet,51.645535,-0.174664
4,Hendon Prep Preschool,Barnet,51.591068,-0.219317
5,Lubavitch of Edgware Kindergarten,Barnet,51.618344,-0.271386
6,Nagila Preschool,Barnet,51.618865,-0.290961
7,The Four Seasons Montessori,Barnet,51.616410,-0.234389
8,The Wood Street Montessori,Barnet,51.653259,-0.206563
9,Alban Pre-school,Barnet,51.658763,-0.206912


In [4]:
address = 'London,UK'

geolocator = Nominatim(user_agent="uk_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


I'm creating a map of London with the Pre-schools superimposed on it.

In [5]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, nurseryname in zip(London['Latitude'], London['Longitude'], London['Borough'], London['Nursery Name']):
    label = '{}, {}'.format(nurseryname, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [6]:
CLIENT_ID = 'GGNHWFPUG4BQYY3RP5PVUIPAUXTI32IBP3V4VBUN054HGKJ0' 
CLIENT_SECRET = 'N1KFLSJIUL0WNO53X0MTVCO4PJ4X5JZ0ISVGWHLLSQECSRLH' 
VERSION = '20180605' 
LIMIT = 100 

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: GGNHWFPUG4BQYY3RP5PVUIPAUXTI32IBP3V4VBUN054HGKJ0
CLIENT_SECRET:N1KFLSJIUL0WNO53X0MTVCO4PJ4X5JZ0ISVGWHLLSQECSRLH


In [7]:
London.loc[7,'Nursery Name']

'The Four Seasons Montessori'

In [8]:
latit = London.loc[7, 'Latitude'] 
longit = London.loc[7, 'Longitude'] 

nursery_name = London.loc[7, 'Borough'] 

print('Latitude and longitude values of {} are {}, {}.'.format(nursery_name, 
                                                               latit,longit))

Latitude and longitude values of Barnet   are 51.6164104, -0.23438910000000002.


In [9]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latit, longit, VERSION, radius, LIMIT)
#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&LL={}&radius={}&limit={}'.format(
#CLIENT_ID, CLIENT_SECRET, VERSION, latit,longitude, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/search?client_id=GGNHWFPUG4BQYY3RP5PVUIPAUXTI32IBP3V4VBUN054HGKJ0&client_secret=N1KFLSJIUL0WNO53X0MTVCO4PJ4X5JZ0ISVGWHLLSQECSRLH&ll=51.6164104,-0.23438910000000002&v=20180605&radius=500&limit=100'

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fbaba1dd64a734725b33979'},
 'response': {'venues': [{'id': '5712741b498e20f5ec91a6f7',
    'name': 'Tesco Express',
    'location': {'address': '59-61 Mill Hill Broadway',
     'lat': 51.614986337257996,
     'lng': -0.24670513895443946,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.614986337257996,
       'lng': -0.24670513895443946}],
     'distance': 865,
     'postalCode': 'NW7 3DA',
     'cc': 'GB',
     'city': 'Mill Hill',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ['59-61 Mill Hill Broadway',
      'Mill Hill',
      'Greater London',
      'NW7 3DA',
      'United Kingdom']},
    'categories': [{'id': '4bf58dd8d48988d118951735',
      'name': 'Grocery Store',
      'pluralName': 'Grocery Stores',
      'shortName': 'Grocery Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      'primary': True}],
    'refe

In [11]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,5712741b498e20f5ec91a6f7,Tesco Express,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",v-1606072861,False,59-61 Mill Hill Broadway,51.614986,-0.246705,"[{'label': 'display', 'lat': 51.61498633725799...",865,NW7 3DA,GB,Mill Hill,Greater London,United Kingdom,"[59-61 Mill Hill Broadway, Mill Hill, Greater ...",NaN,NaN
1,4b9cfc7cf964a520b08636e3,Mill Hill Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1606072861,False,Daws Ln.,51.612261,-0.239167,"[{'label': 'display', 'lat': 51.61226060593113...",567,NW 7 4,GB,Mill Hill,Greater London,United Kingdom,"[Daws Ln., Mill Hill, Greater London, NW 7 4, ...",NaN,NaN
2,4e2812c1a809ec0663e9f211,Mill Hill School Foundation,"[{'id': '4d4b7105d754a06372d81259', 'name': 'C...",v-1606072861,False,NaN,51.618401,-0.230266,"[{'label': 'display', 'lat': 51.61840117440904...",360,NaN,GB,Mill Hill,Greater London,United Kingdom,"[The Ridgeway, Mill Hill, Greater London, Unit...",The Ridgeway,NaN
3,577371ac498e0d1ef45dc897,Belmont Mill Hill Preparatory School,"[{'id': '4f4533814b9074f6e4fb0106', 'name': 'M...",v-1606072861,False,NaN,51.623113,-0.235362,"[{'label': 'display', 'lat': 51.62311267319092...",749,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,NaN
4,4b921218f964a520e4e533e3,mill hill school,"[{'id': '4bf58dd8d48988d1ab941735', 'name': 'S...",v-1606072861,False,NaN,51.619408,-0.230907,"[{'label': 'display', 'lat': 51.61940750461308...",411,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,NaN


In [12]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Tesco Express,Grocery Store,59-61 Mill Hill Broadway,51.614986,-0.246705,"[{'label': 'display', 'lat': 51.61498633725799...",865,NW7 3DA,GB,Mill Hill,Greater London,United Kingdom,"[59-61 Mill Hill Broadway, Mill Hill, Greater ...",NaN,5712741b498e20f5ec91a6f7
1,Mill Hill Park,Park,Daws Ln.,51.612261,-0.239167,"[{'label': 'display', 'lat': 51.61226060593113...",567,NW 7 4,GB,Mill Hill,Greater London,United Kingdom,"[Daws Ln., Mill Hill, Greater London, NW 7 4, ...",NaN,4b9cfc7cf964a520b08636e3
2,Mill Hill School Foundation,College & University,NaN,51.618401,-0.230266,"[{'label': 'display', 'lat': 51.61840117440904...",360,NaN,GB,Mill Hill,Greater London,United Kingdom,"[The Ridgeway, Mill Hill, Greater London, Unit...",The Ridgeway,4e2812c1a809ec0663e9f211
3,Belmont Mill Hill Preparatory School,Middle School,NaN,51.623113,-0.235362,"[{'label': 'display', 'lat': 51.62311267319092...",749,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,577371ac498e0d1ef45dc897
4,mill hill school,Student Center,NaN,51.619408,-0.230907,"[{'label': 'display', 'lat': 51.61940750461308...",411,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,4b921218f964a520e4e533e3
5,Kurtinz.com,Furniture / Home Store,Daws House,51.616591,-0.241764,"[{'label': 'display', 'lat': 51.61659095501365...",510,NW7 4SD,GB,London,Greater London,United Kingdom,"[Daws House (33-35 Daws Lane), London, Greater...",33-35 Daws Lane,501253a5011c39fed8cb8b60
6,Beauty By Design,Spa,NaN,51.612026,-0.231970,"[{'label': 'display', 'lat': 51.61202599199608...",515,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,53384c3c498e75abcd76000f
7,Domino's Pizza,Pizza Place,"127 The Broadway, Mill Hill",51.616202,-0.245110,"[{'label': 'display', 'lat': 51.6162015, 'lng'...",741,NW7 3TJ,GB,London,Greater London,United Kingdom,"[127 The Broadway, Mill Hill, London, Greater ...",NaN,5cbf4667c4df1d002cd64e28
8,Anytime Fitness,Gym / Fitness Center,1 Flower Ln,51.615356,-0.244835,"[{'label': 'display', 'lat': 51.615356, 'lng':...",731,NW7 2JA,GB,London,Greater London,United Kingdom,"[1 Flower Ln, London, Greater London, NW7 2JA,...",NaN,56706a3e498e18cc0bd7d08c
9,Three Hammers,Pub,The Ridgeway,51.621593,-0.234008,"[{'label': 'display', 'lat': 51.62159307322067...",577,NW7 4EA,GB,London,Greater London,United Kingdom,"[The Ridgeway, London, Greater London, NW7 4EA...",NaN,4b26b41df964a520c67f24e3


In [13]:
dataframe_filtered.name

0                                Tesco Express
1                               Mill Hill Park
2                  Mill Hill School Foundation
3         Belmont Mill Hill Preparatory School
4                             mill hill school
5                                  Kurtinz.com
6                             Beauty By Design
7                               Domino's Pizza
8                              Anytime Fitness
9                                Three Hammers
10                                PizzaExpress
11                                      Prezzo
12                                   Pizza Hut
13                 St Paul's CE Primary School
14                                The Reddings
15                      Mill Hill Sport Centre
16                              Brookfield Ave
17                                 Burton Bank
18                               Mani & Nick’s
19                                       Leyla
20                                The Creamery
21           

In [31]:
venues_map = folium.Map(location=[latit, longit], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latit, longit],
    radius=10,
    color='red',
    popup='The Four Seasons Montessori',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=name,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map